# BibRec

### RF Features: Country, State, Age, Year-of-Publication

In [18]:
from bibrec.server.Utils import get_books, get_users, get_ratings

books = get_books("data/BX-Books.csv")
users = get_users("data/BX-Users.csv")
ratings = get_ratings("data/BX-Book-Ratings.csv")

/run/media/fab/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(path, sep=";", encoding="latin-1")


With NaN values 34.862889904962536
used mean values 34.87993394676011


/run/media/fab/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:38: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [20]:
explicit_ratings = ratings[ratings.book_rating != 0]
explicit_ratings

,user_id,isbn,book_rating,isbn13
1,276726,0155061224,5,9780155061224
3,276729,052165615X,3,9780521656153
4,276729,0521795028,6,9780521795029
6,276736,3257224281,8,9783257224283
7,276737,0600570967,6,9780600570967
...,...,...,...,...
1149771,276704,0743211383,7,9780743211383
1149773,276704,0806917695,5,9780806917696
1149775,276704,1563526298,9,9781563526299
1149777,276709,0515107662,10,9780515107661


In [21]:
df_ratings = explicit_ratings.groupby('isbn13').user_id.count().sort_values(ascending=False)
df_ratings = df_ratings[:1000]
df_ratings = df_ratings.reset_index()
# user_id => nr of ratings
df_ratings = df_ratings.drop('user_id',axis=1)
df_ratings

,isbn13
0,9780316666343


In [22]:
df_ratings = df_ratings.merge(explicit_ratings, on='isbn13', how='left')
df_ratings

,isbn13,user_id,isbn,book_rating
0,9780316666343,277439,0316666343,7
1,9780316666343,277752,0316666343,7
2,9780316666343,278633,0316666343,9
3,9780316666343,638,0316666343,10
4,9780316666343,709,0316666343,3
...,...,...,...,...
706,9780316666343,275933,0316666343,9
707,9780316666343,276014,0316666343,5
708,9780316666343,276545,0316666343,9
709,9780316666343,276670,0316666343,10


In [23]:
# merge book dataset
df = df_ratings.merge(books[['isbn13', 'year_of_publication']])
df

,isbn13,user_id,isbn,book_rating,year_of_publication
0,9780316666343,277439,0316666343,7,2002.0
1,9780316666343,277752,0316666343,7,2002.0
2,9780316666343,278633,0316666343,9,2002.0
3,9780316666343,638,0316666343,10,2002.0
4,9780316666343,709,0316666343,3,2002.0
...,...,...,...,...,...
706,9780316666343,275933,0316666343,9,2002.0
707,9780316666343,276014,0316666343,5,2002.0
708,9780316666343,276545,0316666343,9,2002.0
709,9780316666343,276670,0316666343,10,2002.0


In [24]:
# merge user dataset
df = df_ratings.merge(users[['user_id', 'country', 'state', 'age']])
df

,isbn13,user_id,isbn,book_rating,country,state,age
0,9780316666343,277439,0316666343,7,usa,oregon,65.000000
1,9780316666343,277752,0316666343,7,usa,louisiana,18.000000
2,9780316666343,278633,0316666343,9,usa,utah,35.705755
3,9780316666343,638,0316666343,10,usa,california,20.000000
4,9780316666343,709,0316666343,3,usa,virginia,14.000000
...,...,...,...,...,...,...,...
706,9780316666343,275933,0316666343,9,usa,kansas,29.000000
707,9780316666343,276014,0316666343,5,usa,california,28.922584
708,9780316666343,276545,0316666343,9,NaN,overijssel,38.000000
709,9780316666343,276670,0316666343,10,canada,ontario,30.000000


In [25]:
country = pd.get_dummies(df['country'],drop_first=True)
country

,"cambridgeshire, united kingdom",canada,china,finland,france,germany,iran,italy,malaysia,mexico,...,netherlands,philippines,portugal,singapore,south korea,spain,switzerland,united kingdom,us,usa
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
709,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
state = pd.get_dummies(df['state'],drop_first=True)
state

,alabama,alaska,alberta,alentejo,ap,arizona,arkansas,baden-württemberg,barcelona,barreiro,...,vermont,victoria,virginia,wales,washington,west virginia,western australia,wisconsin,wyoming,zaragoza
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df = pd.concat([df, country, state], axis = 1)
df

,isbn13,user_id,isbn,book_rating,country,state,age,"cambridgeshire, united kingdom",canada,china,...,vermont,victoria,virginia,wales,washington,west virginia,western australia,wisconsin,wyoming,zaragoza
0,9780316666343,277439,0316666343,7,usa,oregon,65.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9780316666343,277752,0316666343,7,usa,louisiana,18.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9780316666343,278633,0316666343,9,usa,utah,35.705755,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9780316666343,638,0316666343,10,usa,california,20.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9780316666343,709,0316666343,3,usa,virginia,14.000000,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,9780316666343,275933,0316666343,9,usa,kansas,29.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707,9780316666343,276014,0316666343,5,usa,california,28.922584,0,0,0,...,0,0,0,0,0,0,0,0,0,0
708,9780316666343,276545,0316666343,9,NaN,overijssel,38.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
709,9780316666343,276670,0316666343,10,canada,ontario,30.000000,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
df = df.loc[:,~df.columns.duplicated()]
df

,isbn13,user_id,isbn,book_rating,country,state,age,"cambridgeshire, united kingdom",canada,china,...,vermont,victoria,virginia,wales,washington,west virginia,western australia,wisconsin,wyoming,zaragoza
0,9780316666343,277439,0316666343,7,usa,oregon,65.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9780316666343,277752,0316666343,7,usa,louisiana,18.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9780316666343,278633,0316666343,9,usa,utah,35.705755,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9780316666343,638,0316666343,10,usa,california,20.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9780316666343,709,0316666343,3,usa,virginia,14.000000,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,9780316666343,275933,0316666343,9,usa,kansas,29.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707,9780316666343,276014,0316666343,5,usa,california,28.922584,0,0,0,...,0,0,0,0,0,0,0,0,0,0
708,9780316666343,276545,0316666343,9,NaN,overijssel,38.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
709,9780316666343,276670,0316666343,10,canada,ontario,30.000000,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df = df.drop(['isbn', 'isbn13', 'user_id', 'country', 'state'], axis = 1)
df

,book_rating,age,"cambridgeshire, united kingdom",canada,china,finland,france,germany,iran,italy,...,vermont,victoria,virginia,wales,washington,west virginia,western australia,wisconsin,wyoming,zaragoza
0,7,65.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,18.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,35.705755,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10,20.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,14.000000,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,9,29.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707,5,28.922584,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
708,9,38.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
709,10,30.000000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
X = df.drop('book_rating', axis = 1) #Features
X

,age,"cambridgeshire, united kingdom",canada,china,finland,france,germany,iran,italy,malaysia,...,vermont,victoria,virginia,wales,washington,west virginia,western australia,wisconsin,wyoming,zaragoza
0,65.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,35.705755,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14.000000,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,29.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
707,28.922584,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
708,38.000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
709,30.000000,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
Y = df['book_rating']   # Target Variables
Y

0       7
1       7
2       9
3      10
4       3
       ..
706     9
707     5
708     9
709    10
710     7
Name: book_rating, Length: 711, dtype: int64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

In [35]:
rfc = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf= 0)
rfc.fit(X_train, y_train)

RandomForestClassifier(min_weight_fraction_leaf=0)

In [40]:
rfc_pred = rfc.predict(X_test)
print(classification_report(y_test,rfc_pred))
print(accuracy_score(y_test,rfc_pred)*100)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         9
           6       0.00      0.00      0.00         6
           7       0.31      0.25      0.28        40
           8       0.29      0.25      0.27        64
           9       0.25      0.26      0.25        47
          10       0.21      0.26      0.23        42

    accuracy                           0.23       214
   macro avg       0.11      0.10      0.10       214
weighted avg       0.24      0.23      0.23       214

22.897196261682243


/home/fab/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fab/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fab/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
